In [79]:
import tensorflow.compat.v1 as tf

# to handle Attempting to capture an EagerTensor without building a function
tf.disable_v2_behavior() 
tf.__version__

import mnist

ModuleNotFoundError: No module named 'mnist'

In [72]:
mnist = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist
x_train.num_examples

AttributeError: 'numpy.ndarray' object has no attribute 'num_examples'

In [9]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

In [58]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [59]:
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden,n_hidden])),
    "wo"  : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out]))
}

In [60]:
weights['wo'].shape

TensorShape([Dimension(1024), Dimension(10)])

In [61]:
def conv(x,weights,bias,stride=1):
    out = tf.nn.conv2d(x,weights,padding="SAME",strides = [1,stride,stride,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [87]:
def cnn(x,weights,biases,keep_prob):
    x = tf.reshape(x,shape = [-1,input_height,input_width,input_channels])
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv1)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']), biases['bh1'])
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    hidden_output_before_dropout = tfn.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout,keep_prob)
    output = tf.add(tf.matmul(hidden_output,weights['wo']), biases['bo'])
    return output

In [63]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
pred = cnn(x,weights,biases)

In [66]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [67]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [76]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [82]:
x_train.shape,y_train.shape

((60000, 28, 28), (60000,))

In [84]:
x_train = x_train.reshape(60000,28*28)
x_train.shape

(60000, 784)

In [ ]:
y_train = y_train.reshape(60000)

In [86]:
len(x_train)

60000

In [85]:
batch_size = 100
for i in range(25):
    num_batches = int(len(x_train)/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x = x_train[j*batch_size:j*batch_size+batch_size]
        batch_y = y_train[j*batch_size:j*batch_size+batch_size]
#         batch_x, batch_y = mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y})
        total_cost += c
    print(total_cost)

ValueError: Cannot feed value of shape (100,) for Tensor Placeholder_9:0, which has shape (?, 10)